<font size="+1" color="#108fff" face="Arial">1. Создать код на Python, который загрузит на локальный компьютер данные о котировках заданных пар криптовалют.</font> 

In [5]:
#Базовые библиотеки
import pandas as pd
import numpy as np
from datetime import datetime

<font color="#108fff" face="Arial">1.1. Задать <br>-список криптовалютных пар; <br>-начальную дату для загрузки;<br>-интервал.</font>

In [73]:
pairs = ['BTC/USDT', 'ETH/USDT', 'SOL/USDT', 'XRP/USDT'] #,'FDUSD/USDT'
Stdate = datetime(2024, 8, 8)
Edate = datetime(2024, 8, 23)
Datediff = Edate - Stdate
limit = abs(Datediff.days) + 1
period = '1d'

<font color="#108fff" face="Arial">1.2. Получить исторические данные по выбранным парам.</font>

In [9]:
!pip install ccxt -qq
import ccxt

In [11]:
# Установить биржу для исторических данных.
exchange = ccxt.binance()

In [83]:
data = []
# Установить точку отсчета в мс
since_timestamp = int(Stdate.timestamp() * 1000)
# Запросить данные для каждой пары
for pair in pairs:
    print(f"Fetching data for {pair} since {Stdate}...")
    ohlcv = exchange.fetch_ohlcv(pair, period, since=since_timestamp, limit=limit)
    for candle in ohlcv:
        data.append({
            'pair': pair,
            'timestamp': candle[0],
            #'open': candle[1],
            #'high': candle[2],
            #'low': candle[3],
            'close': candle[4],
           # 'volume': candle[5]
        })
crypto = pd.DataFrame(data)
# Конвертировать мс в дату
crypto['date'] = pd.to_datetime(crypto['timestamp'], unit='ms')

Fetching data for BTC/USDT since 2024-08-08 00:00:00...
Fetching data for ETH/USDT since 2024-08-08 00:00:00...
Fetching data for SOL/USDT since 2024-08-08 00:00:00...
Fetching data for XRP/USDT since 2024-08-08 00:00:00...
Fetching data for FDUSD/USDT since 2024-08-08 00:00:00...


<font color="#108fff" face="Arial">1.3. Преобразовать таблицу к виду по аналогии с акциями.</font>

In [ ]:
crypto = crypto.pivot(index='date', columns='pair', values='close')

<font color="red" face="Arial">Далее используется код из акций.</font><br>
<font color="#108fff" face="Arial">1.4. Обработать пропуски.</font>

In [101]:
#Установить порог данных с Null
threshold = len(crypto) * 0.5
#Удалить колонки за порогом
crypto = crypto.dropna(axis=1, thresh=threshold)
#Заменить пропуски последним значением
crypto = crypto.ffill()
#Заменить пропуски ближайшим следующим значением
crypto = crypto.bfill()

<font color="#108fff" face="Arial">1.5. Выгрузить данные в файл.</font>

In [106]:
filename = f'crypto_{period}_{Stdate.strftime("%Y%m%d")}_{Edate.strftime("%Y%m%d")}.csv'
crypto.to_csv(filename, header=True, index=True)

<font size="+1" color="#108fff" face="Arial">2. Подготовить автоматическое отчеты.</font>

In [115]:
!pip install sweetviz -qq
import sweetviz as sv

In [117]:
report = sv.analyze(crypto)
report.show_html('crypto_report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report crypto_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [119]:
!pip install dtale -qq
import dtale
d = dtale.show(crypto)
d.open_browser()